In [ ]:
import os
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import datetime
from typing import Tuple

global DATA_DEB
global IMAB
global REUNE
global TAXAS_IPCA
global TAXAS_CDI
global TAXAS_PCT_CDI
global ETTJ
global RATING
global CLASS_DEB
global BDINFRA_path
global date

def clear_sheet(sheet: str) -> None:
    """
        Função para limpar a sheet por completo

        :param sheet: Nome da string para ser limpada no workbook
    """

    global BDINFRA_path
                                                                           
    wb = openpyxl.load_workbook(BDINFRA_path)                                       
    ws = wb[sheet]

    for idx_row, row in enumerate(ws, 1): # Limpando cada célula da sheet
        for col in row:
            col.value = None
    
    wb.save(BDINFRA_path) #Salva as alterações

def pandas_to_excel(sheet: str, df: pd.DataFrame) -> None:
    """
        Realizar a transição de pandas DataFrame para planilha em Excel

        :param sheet: Nome da string para ser populada no workbook
        :param df: DataFrame para ser usada para popular a sheet
    """
    global BDINFRA_path

    wb = openpyxl.load_workbook(BDINFRA_path)                                       
    ws = wb[sheet]

    rows = dataframe_to_rows(df, index=False) #Transforma o dataframe em várias células

    for r_idx, row in enumerate(rows, 1): # Realiza o preenchimento da sheet
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    wb.save(BDINFRA_path) #Salva as alterações

def date_to_file(dt: str, type: str) -> str:
    """
        Função para converter a data em nome dos files

        :param dt: Data a ser analisada
        :param type: Para qual documento será feito o ajuste
    """

    dt = dt.split('/') # Retira os chars /

    # Criação dos nome dos files de acordo com a data o tipo passado
    if (type == 'IPCA') or (type == 'CDI') or (type == '%CDI'): 
        mes = {1: 'jan', 2: 'fev', 3: 'mar', 4: 'abr', 5: 'mai', 6: 'jun', 7: 'jul', 8: 'ago', 9: 'set', 10: 'out', 11: 'nov', 12: 'dez'}
        file = 'd'+dt[2][-2:]+mes[int(dt[1])]+dt[0]+'.xls'

    if type == 'REUNE':
        file = 'REUNE_Acumulada_'+''.join(dt)+'.csv'

    if type == 'IMAB':
        file = 'IMA_'+''.join(dt)+'.csv'

    if type == 'ETTJ':
        file = 'CurvaZero_'+''.join(dt)+'.csv'
    
    return file # Retorna o nome do file

def clean(df: pd.DataFrame , type: str) -> Tuple[pd.DataFrame, datetime.datetime]:
    """
        Realizar pré-processamentos para bases a serem exploradas

        :param df: DataFrame a ser realizado o pré-processamento
        :param type: Para qual documento será feito o pré-processamento
    """

    # Realiza o pré-processamento a depender do tipo
    if (type == 'IPCA') or (type == 'CDI') or (type == '%CDI'):
        col_1 = list(df.columns)[0]
        idx_srt = list(df.index[df[col_1] == 'Código'])[0]
        idx_end = list(df.index[df[col_1].astype(str).str.contains("[*]")])[0]
        columns = pd.Series(list(df.iloc[idx_srt]))
        col_intind = list(columns[columns.astype(str).str.contains('Intervalo Indicativo')])
        if len(col_intind) == 1:
            columns = columns.fillna('Intervalo Indicativo Máxima')
            columns = columns.replace(col_intind[0], "Intervalo Indicativo Mínimo")
        df.columns = columns
        df = df.iloc[idx_srt+2:idx_end]
        df = df.reset_index(drop=True)
        df['Nome'] = [name[:name.index('*')-2] if '*' in name else name for name in df['Nome']]
        df['Nome'] = [name[:name.index('#')-2] if '#' in name else name for name in df['Nome']]

    if type == 'REUNE':
        df = df.reset_index()
        col_1 = list(df.columns)[0]
        idx_srt = list(df.index[df[col_1].astype(str).str.contains('CETIP')])[0]
        columns = pd.Series(list(df.iloc[idx_srt]))
        df.columns = columns
        df = df.iloc[idx_srt+1:]
        df = df.reset_index(drop=True)
        df = df[df['Agrupamento'] != 'SUBTOTAL']
        df['Taxa Média'] = df['Taxa Média'].str.replace(',', '.')
        df['Preço Médio'] = df['Preço Médio'].str.replace('.', '')
        df['Preço Médio'] = df['Preço Médio'].str.replace(',', '.')

    if (type == 'TAXAS') or (type == 'ETTJ') or (type == 'IMAB'):
        df.rename(columns={ df.columns[0]: "Tickers", df.columns[1]: "Tipo"}, inplace = True)
        df.loc[:, ['Tickers']] = df.loc[:, ['Tickers']].ffill(axis=0)
        df = df.set_index(['Tickers', 'Tipo'])

    if type == 'DATA_DEB':
        df['Codigo do Ativo'] = df['Codigo do Ativo'].str.replace(' ','')
        df.columns = df.columns.str.replace(' ','')

    dt_rat = None
    if type == 'RATING':
        dt_rat = pd.to_datetime(df.columns[0])
        df.columns = df.iloc[0]
        df = df.iloc[1:]

    return df, dt_rat # Retorna o DataFrame já ajustado para análises

def source(dt: str):
    """
        Pegar base de dados crua

        :param str: Data para ser analisada
    """
    
    downloads_path = "~\Downloads"
    downloads_path = os.path.expanduser(downloads_path)

    global date
    date = dt

    global DATA_DEB
    DATA_DEB = pd.read_table(os.path.join(downloads_path, "Debentures.com.br_Caracteristica_em_03-03-2021_as_21-19-42.xls"), encoding='ANSI')
    DATA_DEB, Lixo  = clean(DATA_DEB, 'DATA_DEB')

    global REUNE
    REUNE = pd.read_csv(os.path.join(downloads_path, date_to_file(date, 'REUNE - 2019')))
    REUNE, Lixo = clean(REUNE, 'REUNE')

    global IMAB
    IMAB = pd.read_csv(os.path.join(downloads_path, date_to_file(date, 'IMAB - 2019')))
    IMAB, Lixo  = clean(IMAB, 'IMAB')

    global TAXAS
    TAXAS = pd.read_excel(os.path.join(downloads_path, "Taxas - 2019.xlsx"))
    TAXAS, Lixo  = clean(TAXAS, 'TAXAS')

    global ETTJ
    ETTJ = pd.read_csv(os.path.join(downloads_path, date_to_file(date, 'ETTJ')),sep=";", encoding='ANSI')
    ETTJ, Lixo = clean(ETTJ, 'ETTJ')

    global RATING
    RATING = pd.read_excel(os.path.join(downloads_path, "Rating.xlsx"))
    RATING, dt_rate = clean(RATING, 'RATING')

    global CLASS_DEB
    CLASS_DEB = pd.read_excel(os.path.join(downloads_path, "Classificação Debêntures.xlsx"))

    global BDINFRA_path 
    BDINFRA_path = os.path.join(downloads_path, "BD Infra - Secundário.xlsx")

    return dt_rate